### Libraries and datasets


In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from scipy.stats import shapiro, skew, kurtosis

red   = pd.read_csv('winequality-red.csv',   sep=';')
white = pd.read_csv('winequality-white.csv', sep=';')


### Normality test graphically

In [ ]:
for k in [red, white]:
    label = 'red' if k is red else 'white'
    variables= k.select_dtypes(include='number').columns
    n = len(variables) 
    ncols = 4
    nrows = 3
    fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(8*ncols, 6*nrows))
    axes = axes.flatten()

    for i, col in enumerate(variables):
        series = k[col]
        ax=axes[i]
        mean      = series.mean()        
        std       = series.std()        
        skewness  = series.skew()        
        kurtosis  = series.kurt()        
        q1        = series.quantile(0.25)
        median    = series.quantile(0.50)
        q3        = series.quantile(0.75)
        iqr = q3 - q1
        upper_limit = q3 + 1.5 * iqr
        lower_limit = q1 - 1.5 * iqr
        ax.axvline(upper_limit, color='brown', linestyle='-', linewidth=4, label='+IQR')
        ax.axvline(lower_limit, color='brown', linestyle='-', linewidth=4, label='-IQR ')

        ax.hist(series, bins=150, edgecolor='black')

        ax.axvline(mean, color='red', linestyle='dashed', linewidth=2, label=f'Mean')
        ax.axvline(mean + 3*std, color='green', linestyle=':', linewidth=2, label='+3sigma')
        ax.axvline(mean - 3*std, color='green', linestyle=':', linewidth=2, label='-3sigma')
        
        stats_str = ( f"Mean: {mean:.4f}\n" f"Std:  {std:.4f}\n" f"Skew: {skewness:.4f}\n" f"Kurt: {kurtosis:.4f}")

        ax.text(0.95, 0.95,stats_str,ha='right', va='top',transform=ax.transAxes, bbox=dict(facecolor='white', alpha=0.7, pad=5))

        ax.set_title(f'Histogram of {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Frequency')
        ax.legend()
        plt.savefig(f'{label}histograms.png',dpi=800)
    plt.show()

### Normality test analitically

In [2]:
names= red.select_dtypes(include='number').columns #Works with both dataframes as they have the same variables
names = [col for col in names if col != 'quality'] #Quality is not a variable to be analyzed here


emptylist=[]
for i in names:
    skewness_red = red[i].skew()
    kurtosis_red = red[i].kurt() 
    skewness_white = white[i].skew()
    kurtosis_white = white[i].kurt()
    emptylist.append([skewness_red, kurtosis_red, skewness_white, kurtosis_white])

normality= pd.DataFrame(emptylist, columns=['Skewness Red', 'Kurtosis Red', 'Skewness White', 'Kurtosis White'], index=names)

normality.reset_index(inplace=True)
normality.rename(columns={'index': 'Variable'}, inplace=True)


# Highlight skewness columns
styled = normality.style.map(
    lambda x: 'background-color: red; color: white;' if abs(x) > 1 else '',
    subset=['Skewness Red', 'Skewness White']
)

# Highlight kurtosis columns
styled = styled.map(
    lambda x: 'background-color: red; color: white;' if abs(x) > 2 else '',
    subset=['Kurtosis Red', 'Kurtosis White']
)

styled

,Variable,Skewness Red,Kurtosis Red,Skewness White,Kurtosis White
0,fixed acidity,0.982751,1.132143,0.647751,2.172178
1,volatile acidity,0.671593,1.225542,1.576980,5.091626
2,citric acid,0.318337,-0.788998,1.281920,6.174901
3,residual sugar,4.540655,28.617595,1.077094,3.469820
4,chlorides,5.680347,41.715787,5.023331,37.564600
5,free sulfur dioxide,1.250567,2.023562,1.406745,11.466342
6,total sulfur dioxide,1.515531,3.809824,0.390710,0.571853
7,density,0.071288,0.934079,0.977773,9.793807
8,pH,0.193683,0.806943,0.457783,0.530775
9,sulphates,2.428672,11.720251,0.977194,1.590930


### Variables with Outliers

In [3]:
outlier_red={}
outlier_white={}

for i,row in normality.iterrows():
    var=row['Variable'] # i is the index of the row, row is the row itself, 
                    #then,row its the name of each variable   

    
    #Now, a condition for non-normality for red wine              
    if abs(row['Skewness Red']) > 1 or abs(row['Kurtosis Red']) > 2:
        outlier_red[var]= 'IQR'
    #So the others are the normal distributions, where we use Z-score
    else:
        outlier_red[var]= 'Z-score'    

    #Now, a condition for non-normality for white wine   
    if abs(row['Skewness White']) > 1 or abs(row['Kurtosis White']) > 2:
        outlier_white[var]= 'IQR'
    #So the others are the normal distributions, where we use Z-score
    else:
        outlier_white[var]= 'Z-score'  
print('Outliers Red:', outlier_red)
print('Outliers White:', outlier_white)


Outliers Red: {'fixed acidity': 'Z-score', 'volatile acidity': 'Z-score', 'citric acid': 'Z-score', 'residual sugar': 'IQR', 'chlorides': 'IQR', 'free sulfur dioxide': 'IQR', 'total sulfur dioxide': 'IQR', 'density': 'Z-score', 'pH': 'Z-score', 'sulphates': 'IQR', 'alcohol': 'Z-score'}
Outliers White: {'fixed acidity': 'IQR', 'volatile acidity': 'IQR', 'citric acid': 'IQR', 'residual sugar': 'IQR', 'chlorides': 'IQR', 'free sulfur dioxide': 'IQR', 'total sulfur dioxide': 'Z-score', 'density': 'IQR', 'pH': 'Z-score', 'sulphates': 'Z-score', 'alcohol': 'Z-score'}


### Outlier detection

In [4]:
outlier_indices_remove_red = {}
outlier_indices_cap_red = {}
outlier_indices_remove_white = {}
outlier_indices_cap_white = {}

for w in names:
    method_red = outlier_red[w] #this has IQR or Z-score (defined above)
    variable_col_red = red[w] #this has the column of the variable
    # then, we are assigning the method to each variable
    if method_red == 'IQR':
        q1 = variable_col_red.quantile(0.25)
        q3 = variable_col_red.quantile(0.75)
        iqr = q3 - q1
        upper_red = q3 + 1.5 * iqr
        lower_red = q1 - 1.5 * iqr
        outliers_red = variable_col_red[(variable_col_red < lower_red) | (variable_col_red > upper_red)].index.tolist()

    else:
        mean_red = variable_col_red.mean()
        std_dev_red = variable_col_red.std()
        z_score_red = (variable_col_red - mean_red) / std_dev_red
        outliers_red = variable_col_red[(z_score_red < -3) | (z_score_red > 3)].index.tolist()
    outlier_indices_remove_red[w] = outliers_red 
    outlier_indices_cap_red[w] = outliers_red   
    #print(f"{w}: {len(outlier_indices_remove_red[w])} outliers")

# Same as before, but for white
    method_white = outlier_white[w] 
    variable_col_white = white[w]      
   
    if method_white == 'IQR':                           
        q1 = variable_col_white.quantile(0.25)
        q3 = variable_col_white.quantile(0.75)
        iqr = q3 - q1
        upper_white = q3 + 1.5 * iqr
        lower_white = q1 - 1.5 * iqr
        outliers_white = variable_col_white[(variable_col_white < lower_white) | (variable_col_white > upper_white)].index.tolist()
    else:
        mean_white = variable_col_white.mean()
        std_dev_white = variable_col_white.std()
        z_score_white = (variable_col_white - mean_white) / std_dev_white
        outliers_white = variable_col_white[(z_score_white < -3) | (z_score_white > 3)].index.tolist()
    outlier_indices_remove_white[w] = outliers_white
    outlier_indices_cap_white[w] = outliers_white
    #print(f"{w}: {len(outlier_indices_remove_white[w])} outliers")


### Option A: Removing Outliers

In [5]:
#Now, let's create a DataFrame with the indexes of the outliers without repeating

outlier_indices_all_red = set()
for indices in outlier_indices_remove_red.values():
    outlier_indices_all_red.update(indices)
red_removed = red.drop(index=outlier_indices_all_red).reset_index(drop=True)

outlier_indices_all_white = set()
for indices in outlier_indices_remove_white.values():
    outlier_indices_all_white.update(indices)
white_removed = white.drop(index=outlier_indices_all_white).reset_index(drop=True)  

### Option B: Capping Outliers

In [6]:
red_capped = red.copy()
white_capped = white.copy()

for w in names:
    method_red = outlier_red[w]
    variable_col_red = red_capped[w] 
    
    if method_red == 'IQR':
        q1 = variable_col_red.quantile(0.25)
        q3 = variable_col_red.quantile(0.75)
        iqr = q3 - q1
        upper_red = q3 + 1.5 * iqr
        lower_red = q1 - 1.5 * iqr

    else:
        mean_red = variable_col_red.mean()
        std_dev_red = variable_col_red.std()
        upper_red = mean_red + 3 * std_dev_red
        lower_red = mean_red - 3 * std_dev_red
    red_capped[w] = red_capped[w].clip(lower=lower_red, upper=upper_red)  

#Same for white
      

for w in names:
    method_white = outlier_white[w]
    variable_col_white = white_capped[w] 
    
    if method_white == 'IQR':
        q1 = variable_col_white.quantile(0.25)
        q3 = variable_col_white.quantile(0.75)
        iqr = q3 - q1
        upper_white = q3 + 1.5 * iqr
        lower_white = q1 - 1.5 * iqr

    else:
        mean_white = variable_col_white.mean()
        std_dev_white = variable_col_white.std()
        upper_white = mean_white + 3 * std_dev_white
        lower_white = mean_white - 3 * std_dev_white
    white_capped[w] = white_capped[w].clip(lower=lower_white, upper=upper_white)        



### Now, the dataframes I have are

In [7]:
red   = pd.read_csv('winequality-red.csv',   sep=';')
white = pd.read_csv('winequality-white.csv', sep=';')
red_capped 
white_capped
red_removed
white_removed

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4116,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4117,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4118,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4119,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


### Normalization between 0 and 1

In [8]:
def normalizator(df):
    df_normalized = df.copy()
    for col in names:
        min_val = df[col].min()
        max_val = df[col].max()
        df_normalized[col] = (df[col] - min_val) / (max_val - min_val)
    return df_normalized    



red_capped_n = normalizator(red_capped)  
white_capped_n = normalizator(white_capped)
red_removed_n = normalizator(red_removed)
white_removed_n = normalizator(white_removed)
red_capped_n.to_csv('red-capped.csv', sep=';', index=False)
white_capped_n.to_csv('white-capped.csv', sep=';', index=False)
red_removed_n.to_csv('red-removed.csv', sep=';', index=False)
white_removed_n.to_csv('white-removed.csv', sep=';', index=False)
